In [1]:
#data loaders
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.document_loaders import DirectoryLoader


#data chunking 
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_text_splitters import SentenceTransformersTokenTextSplitter #sematic 




#Embedding
from sentence_transformers import SentenceTransformer #embedding model
from langchain_huggingface import HuggingFaceEmbeddings


#VectorDB
import chromadb
# from langchain.vectorstores import Chroma
from langchain_chroma import Chroma

#LLm
from dotenv import load_dotenv 
from langchain_huggingface import HuggingFaceEndpoint
from langchain_core.prompts import PromptTemplate   #system prompt
from langchain_core.prompts import ChatPromptTemplate #system prompt
from langchain_huggingface import ChatHuggingFace
from langchain_huggingface import HuggingFaceEndpoint






d:\pratice_Arena\krish_naik\Rag_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load Document

In [2]:
## load all the pdf files from the directory
dir_loader= DirectoryLoader(
    "../Data/pdf",
    glob="**/*.pdf",  ## Pattern to match files
    loader_cls= PyMuPDFLoader, ##loader class to use
    show_progress=False
    
)

pdf_doc = dir_loader.load()
print(len(pdf_doc))
pdf_doc


203


[Document(metadata={'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2024-04-10T21:11:43+00:00', 'source': '..\\Data\\pdf\\attention.pdf', 'file_path': '..\\Data\\pdf\\attention.pdf', 'total_pages': 15, 'format': 'PDF 1.5', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'moddate': '2024-04-10T21:11:43+00:00', 'trapped': '', 'modDate': 'D:20240410211143Z', 'creationDate': 'D:20240410211143Z', 'page': 0}, page_content='Provided proper attribution is provided, Google hereby grants permission to\nreproduce the tables and figures in this paper solely for use in journalistic or\nscholarly works.\nAttention Is All You Need\nAshish Vaswani∗\nGoogle Brain\navaswani@google.com\nNoam Shazeer∗\nGoogle Brain\nnoam@google.com\nNiki Parmar∗\nGoogle Research\nnikip@google.com\nJakob Uszkoreit∗\nGoogle Research\nusz@google.com\nLlion Jones∗\nGoogle Research\nllion@google.com\nAidan N. Gomez∗†\nUniversity of Toronto\naidan@cs.toronto.edu\nŁukasz Kaiser∗\nGoogle

## Chunking the documents

In [3]:
# def split_documents(documents, chunk_size=1000, chunk_overlap=200):
#     """
#     Split documents into smaller chunks using recursive, structure-aware chunking.
#     """

#     text_splitter = RecursiveCharacterTextSplitter(
#         chunk_size=chunk_size,
#         chunk_overlap=chunk_overlap,
#         length_function=len,
#         separators=["\n\n", "\n", " ", ""]
#     )

#     split_docs = text_splitter.split_documents(documents)

#     print(f"Split {len(documents)} documents into {len(split_docs)} chunks")

#     # Show one example chunk (for demo/debug)
#     if split_docs:
#         print("\nExample chunk:")
#         print(split_docs[0].page_content[:200], "...")
#         print("Metadata:", split_docs[0].metadata)

#     return split_docs

In [4]:
def semantic_split_documents(
    documents,
    embedding_model="sentence-transformers/all-MiniLM-L6-v2",
    chunk_size=500
):
    """
    Semantic chunking using sentence-transformer embeddings.
    Groups semantically similar sentences together.
    """

    splitter = SentenceTransformersTokenTextSplitter(
        model_name=embedding_model,
        chunk_size=chunk_size,
        chunk_overlap=0
    )

    semantic_docs = []

    for doc in documents:
        chunks = splitter.split_text(doc.page_content)

        for chunk in chunks:
            semantic_docs.append(
                doc.__class__(
                    page_content=chunk,
                    metadata=doc.metadata
                )
            )

    print(f"Semantic split of {len(documents)} documents into {len(semantic_docs)} chunks")

    # Example chunk
    if semantic_docs:
        print("\nExample semantic chunk:")
        print(semantic_docs[0].page_content[:200], "...")
        print("Metadata:", semantic_docs[0].metadata)

    return semantic_docs
semantic_chunks = semantic_split_documents(pdf_doc)
semantic_chunks

Semantic split of 203 documents into 585 chunks

Example semantic chunk:
provided proper attribution is provided, google hereby grants permission to reproduce the tables and figures in this paper solely for use in journalistic or scholarly works. attention is all you need  ...
Metadata: {'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2024-04-10T21:11:43+00:00', 'source': '..\\Data\\pdf\\attention.pdf', 'file_path': '..\\Data\\pdf\\attention.pdf', 'total_pages': 15, 'format': 'PDF 1.5', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'moddate': '2024-04-10T21:11:43+00:00', 'trapped': '', 'modDate': 'D:20240410211143Z', 'creationDate': 'D:20240410211143Z', 'page': 0}


[Document(metadata={'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2024-04-10T21:11:43+00:00', 'source': '..\\Data\\pdf\\attention.pdf', 'file_path': '..\\Data\\pdf\\attention.pdf', 'total_pages': 15, 'format': 'PDF 1.5', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'moddate': '2024-04-10T21:11:43+00:00', 'trapped': '', 'modDate': 'D:20240410211143Z', 'creationDate': 'D:20240410211143Z', 'page': 0}, page_content='provided proper attribution is provided, google hereby grants permission to reproduce the tables and figures in this paper solely for use in journalistic or scholarly works. attention is all you need ashish vaswani∗ google brain avaswani @ google. com noam shazeer∗ google brain noam @ google. com niki parmar∗ google research nikip @ google. com jakob uszkoreit∗ google research usz @ google. com llion jones∗ google research llion @ google. com aidan n. gomez∗ † university of toronto aidan @ cs. toronto. edu łukasz kaiser∗ google br

In [5]:
# chunks=split_documents(pdf_doc)
# chunks

## Embedding 

In [6]:
from langchain_huggingface import HuggingFaceEmbeddings

def load_embedding_model(
    model_name: str = "sentence-transformers/all-MiniLM-L6-v2"
):
    """
    Load HuggingFace embedding model using LangChain wrapper
    and verify it is ready.
    """
    try:
        print(f"Loading embedding model: {model_name}")
        
        embeddings = HuggingFaceEmbeddings(
            model_name=model_name
        )
        
        # Simple test to ensure model is loaded
        test_vector = embeddings.embed_query("test sentence")
        
        print(f"Model '{model_name}' loaded successfully")
        print(f"Embedding dimension: {len(test_vector)}")
        
        return embeddings
    
    except Exception as e:
        print(f"Failed to load model '{model_name}'")
        raise e


In [7]:
embeddings = load_embedding_model() 


Loading embedding model: sentence-transformers/all-MiniLM-L6-v2
Model 'sentence-transformers/all-MiniLM-L6-v2' loaded successfully
Embedding dimension: 384


Extract text content from chunks

In [8]:
chunk_texts = [doc.page_content for doc in semantic_chunks]

print(f"Preparing {len(chunk_texts)} chunks for embedding")


Preparing 585 chunks for embedding


Generate embeddings for all chunks

In [9]:
chunk_vectors = embeddings.embed_documents(chunk_texts)

print(f"Generated embeddings for {len(chunk_vectors)} chunks")
print(f"Each embedding dimension: {len(chunk_vectors[0])}")


Generated embeddings for 585 chunks
Each embedding dimension: 384


## Vector Db

In [10]:
# storing in Vector DB

chroma_db = Chroma.from_documents(
    documents=semantic_chunks,
    embedding=embeddings,
    persist_directory="./chroma_store"
)


In [11]:
def retrieve(query, k=4, metadata_filter=None):
    """
    Retrieve relevant chunks using semantic search.
    Optional metadata_filter can be applied if needed.
    """
    if metadata_filter:
        return chroma_db.similarity_search(
            query=query,
            k=k,
            filter=metadata_filter
        )
    return chroma_db.similarity_search(query=query, k=k)

results = retrieve("Explain self-attention")

for doc in results:
    print(doc.page_content[:200])
    print(doc.metadata)


attention visualizations it is in this spirit that a majority of american governments have passed new laws since 2009 making the registration or voting process more difficult. < eos > < pad > < pad > 
{'creator': 'LaTeX with hyperref', 'subject': '', 'moddate': '2024-04-10T21:11:43+00:00', 'keywords': '', 'format': 'PDF 1.5', 'creationdate': '2024-04-10T21:11:43+00:00', 'trapped': '', 'total_pages': 15, 'producer': 'pdfTeX-1.40.25', 'author': '', 'page': 12, 'modDate': 'D:20240410211143Z', 'title': '', 'file_path': '..\\Data\\pdf\\attention.pdf', 'creationDate': 'D:20240410211143Z', 'source': '..\\Data\\pdf\\attention.pdf'}
as noted in table 1, a self - attention layer connects all positions with a constant number of sequentially executed operations, whereas a recurrent layer requires o ( n ) sequential operations. in te
{'modDate': 'D:20240410211143Z', 'moddate': '2024-04-10T21:11:43+00:00', 'title': '', 'creationdate': '2024-04-10T21:11:43+00:00', 'creationDate': 'D:20240410211143Z',

In [12]:
import os

# Load environment variables from .env
load_dotenv()

# Verify token is loaded (optional, safe)
assert os.getenv("HUGGINGFACEHUB_API_TOKEN") is not None, "HF token not found"

In [13]:
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace

hf_endpoint = HuggingFaceEndpoint(
    repo_id="openai/gpt-oss-120b",
    task="conversational",
    temperature=0.2,
    max_new_tokens=400,
)

llm = ChatHuggingFace(llm=hf_endpoint)

# STRICT RAG PROMPT 
RAG_PROMPT = ChatPromptTemplate.from_messages([
    (
        "system",
        "You are an AI assistant who understands the . try to answer best to Question and using provided context. "
        "If the answer is not found, say: "
        "'Information not found in the provided documents.'"
    ),
    (
        "human",
        "Context:\n{context}\n\nQuestion:\n{question}"
    )
])

## Combine retrieval + LLM (core RAG logic)

In [14]:
def answer_query(query, k=5, metadata_filter=None):
    # Step 1: Retrieve chunks
    if metadata_filter:
        docs = chroma_db.similarity_search(
            query=query, k=k, filter=metadata_filter
        )
    else:
        docs = chroma_db.similarity_search(query=query, k=k)

    # Step 2: Build context
    context = "\n\n".join(doc.page_content for doc in docs)

    # Step 3: Build chat messages (THIS IS KEY)
    messages = RAG_PROMPT.format_messages(
        context=context,
        question=query
    )

    # Step 4: Invoke conversational LLM
    answer = llm.invoke(messages)

    return answer, docs


In [18]:
answer, docs = answer_query(
"what is the stock price of nasscom?"
)
print("ANSWER:\n", answer.content)

# print("\nSOURCES:")
# for doc in docs:
#     author = doc.metadata.get("author", "Unknown")
#     page = doc.metadata.get("page", "N/A")
#     source = doc.metadata.get("source", "Unknown")

#     print(f"- {source} | Author: {autpresident  {page}")


ANSWER:
 Information not found in the provided documents.
